In [10]:
# File to pre-process the test set for the Fix few shot learning baseline analysis


from tabulate import tabulate
import pandas as pd
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def table_format_og(df, type):
    num_columns = df.shape[1]
    for i in range(num_columns):
       df.rename(columns={ str(i) : 'Column ' + type + '-' + str(i)}, inplace=True)
    random_rows = df.sample(n=5, random_state=41)
    df_destination = pd.DataFrame(random_rows)
    table_str = tabulate(df_destination, headers='keys', tablefmt='pipe', showindex=False)
    return table_str

def find_nearest_neighbors(df, nearest_neighbor_ind, K=3):


    examples_tables = [] 
    #print(nearest_neighbors)
    for i in nearest_neighbor_ind:
        row = df.iloc[i]
        table_name = row['table_name']
        matching_columns = df_train_og[df_train_og['table_name'] == table_name]
        table_path = WEBTABLES_PATH.format(name=table_name)
        pedia_path = DB_PEDIA_PATH.format(name=table_name)
        table = pd.read_csv(table_path)
        table_A = table_format_og(table, 'A')
        pedia = pd.read_csv(pedia_path)
        table_B = table_format_og(pedia, 'B')
        s = example_format(table_A, table_B, matching_columns)
        examples_tables.append(s)

    return examples_tables
    

def format_answer(matching_columns):
    matching_columns = matching_columns.reset_index(drop=True)
    table_name = matching_columns.iloc[0]['table_name']
    table_path = WEBTABLES_PATH.format(name=table_name)
    table = pd.read_csv(table_path)
    s = '''{"column_mappings": ['''
    for i in range(len(table.columns)):
        marker = False
        for x in range(len(matching_columns)):
            if i == matching_columns.at[x, 'column_index_left'] and  1 == matching_columns.at[x, 'label']:
                j = matching_columns.at[x, 'column_index_right']
                s += f'''["Column A-{i}", "Column B-{j}"],'''
                marker = True
        if not marker:
            s += f'''["Column A-{i}", "None"],'''
    s = s[:-1]
    s += ''']}'''
    return s

def example_format(table_A, table_B, matching_columns):
    return "Input:\nTable A: \n" + table_A + "\n\nTable B: \n" + table_B + "\n\nAnswer: " + format_answer(matching_columns) + "\n\n"
    


DATA_PATH = "../../Datasets/t2d_sm_nh/test/test_correspondences_sample.csv"
TRAIN_PATH = "../../Datasets/t2d_sm_nh/train/train_correspondences.csv"
OUTPUT_PATH = "../../Datasets/t2d_sm_nh/test/test_correspondences_sample_fix_few_shot.csv"
DB_PEDIA_PATH = "../../Datasets/t2d_sm_nh/train/dbpedia_tables/{name}.csv"
WEBTABLES_PATH = "../../Datasets/t2d_sm_nh/train/webtables/{name}.csv"
df = pd.read_csv(DATA_PATH)
df_train = pd.read_csv(TRAIN_PATH)
df_train_og = df_train
df_train = df_train.drop_duplicates(subset='table_name')
df_train['tables'] = ''
df_train.reset_index(drop=True, inplace=True)

df['example_1'] = ''
df['example_2'] = ''





/var/folders/pw/9fnphybj4kq8t6wt3h592_9r0000gn/T/ipykernel_9088/104509237.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['tables'] = ''


In [11]:

for i in range(len(df_train)):
    row = df_train.iloc[i]
    table_name = row['table_name']
    table_path = WEBTABLES_PATH.format(name=table_name)
    pedia_path = DB_PEDIA_PATH.format(name=table_name)
    table = pd.read_csv(table_path)
    table_A = table_format_og(table, 'A')
    pedia = pd.read_csv(pedia_path)
    table_B = table_format_og(pedia, 'B')
    tables_combined = table_A + table_B
    df_train.loc[i, 'tables'] = tables_combined

print(df_train.head())


   Unnamed: 0                       table_name  column_index_left  \
0           0   57938705_0_8737506792349461963                  9   
1          32   12183399_0_6725061928072492226                  0   
2          55   37541068_1_1156932518917623732                  6   
3          72   32812626_0_5428724146873158593                  0   
4          84  29414811_13_8724394428539174350                  4   

   column_index_right  label  \
0                   0      1   
1                   0      1   
2                   0      1   
3                   0      1   
4                   0      1   

                                              tables  
0  | Column A-0   | Column A-1   | Column A-2   |...  
1  | Column A-0         | Column A-1         | Co...  
2  |   Column A-0 |   Column A-1 | Column A-2   |...  
3  | Column A-0                     | Column A-1 ...  
4  |   Column A-0 | Column A-1                   ...  


In [24]:

last = None
nearest_neighbor_ind = df_train.sample(n=2, random_state=9).index.tolist()
for i in range (len(df)):
    row = df.iloc[i]
    if row['table_name'] == last:
        continue    
    last = row['table_name']
    table_A = df.loc[i, 'table_A'] 
    table_B = df.loc[i, 'table_B']
    compare_string = table_A +"\n\n" +  table_B
    #print(compare_string)
    example_tables = find_nearest_neighbors(df_train, nearest_neighbor_ind, K=2)
    matching_columns = df[df['table_name'] == last]
    for x in range(len(example_tables)):
        print(example_tables[x])
        df.loc[i, f'example_{x+1}'] = example_tables[x]
    print(i)
    


for i in example_tables:
    print(i[0])
    print(i[1])


Input:
Table A: 
|   Column A-0 | Column A-1   | Column A-2   | Column A-3   |   Column A-4 | Column A-5   | Column A-6    |
|-------------:|:-------------|:-------------|:-------------|-------------:|:-------------|:--------------|
|           26 | -            | -            | VR-China     |       5694   | -            | Koko-Nor      |
|            7 | 636          | 1680 (2008)  | Russland     |         31.5 | 23.600       | Baikalsee     |
|           55 | -            | -            | Kanada       |       2227   | -            | Yathkyed-Lake |
|           41 | 200          | -            | Mongolei     |       3350   | -            | Ubsa-Nor      |
|           43 | 250          | -            | Usbekistan   |       3000   | 44,3         | Aydar-Ko?l    |

Table B: 
| Column B-0            |   Column B-1 |     Column B-2 |   Column B-3 |   Column B-4 | Column B-5   |   Column B-6 |   Column B-7 |
|:----------------------|-------------:|---------------:|-------------:|-----------

In [25]:
print(df.head())

   table_index                      table_name  column_index_left  \
0            0  29021592_3_2299138476894681059                  0   
1            1  29021592_3_2299138476894681059                  3   
2            2  29021592_3_2299138476894681059                  4   
3            3  29021592_3_2299138476894681059                  2   
4            4  29021592_3_2299138476894681059                  0   

   column_index_right  y_true  y_pred  prompt_name  prompt  \
0                   0    True     NaN          NaN     NaN   
1                   0   False     NaN          NaN     NaN   
2                   0   False     NaN          NaN     NaN   
3                   0   False     NaN          NaN     NaN   
4                   3   False     NaN          NaN     NaN   

                                      column_table_A  \
0        Domitian | Hadrian | Nerva | Titus | Trajan   
1  Titus Flavius Domitianus | Publius Aelius Hadr...   
2              81-96 | 117-8 | 96-8 | 79-81 

In [26]:
print(df['y_true'].value_counts())


y_true
False    371
True      79
Name: count, dtype: int64


In [27]:

# Save df in output path
df.to_csv(OUTPUT_PATH, index=False)